In [ ]:
import requests
import pandas as pd
import io
import csv

import geopandas as gpd
import configparser
from pathlib import Path

In [ ]:
# Config file contains file paths and name of township (uesd to match parcel TWP_CITY column)
config = configparser.ConfigParser()

# Change this path to the location of your own config.ini file
config.read('/home/petermitchell/Documents/ContractWork/config.ini')
config.sections()

# Set variables to inputs from config file
twp = config['DEFAULT']['township']
verified_wells_path = config['DEFAULT']['verified_wells']
unverified_wells_path = config['DEFAULT']['unverified_wells']
parcels_path = config['DEFAULT']['parcels']
samples_path = config['DEFAULT']['samples']
output_path = config['DEFAULT']['output']

output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Upload csv to us census geocoder. As an alternative, you can just go to the url below and upload it yourself
# File cannot exceed 10,000 records
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
files = {'addressFile': (samples_path, open(samples_path, 'rb'), 'text/csv')}
payload = {'benchmark':'Public_AR_Current', 'vintage':'Current_Current'}
print('Posting requests...')
s = requests.post(url, files=files, data=payload)
print(type(s))

In [ ]:
df = pd.read_csv(io.StringIO(s.text), sep=',', header=None, quoting=csv.QUOTE_ALL, keep_default_na=False, names = ['id', 'address_in', 'match_indicator', 'match_type', 'address_out', 'long_lat', 'tiger_edge', 'street_side', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK'])

#df = pd.read_csv('/home/petermitchell/Downloads/GeocodeResults(1).csv', names = ['id', 'address_in', 'match_indicator', 'match_type', 'address_out', 'long_lat', 'tiger_edge', 'street_side', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK'])
pd.DataFrame.head(df)

print(len(df.columns))

In [ ]:
df[['long', 'lat']] = df['long_lat'].str.split(',', expand=True)
out = df[['id', 'long', 'lat', 'address_out']]
out.to_csv(output_dir / '911_addresses_out.csv')


In [ ]:
parcels = gpd.read_file(parcels_path).to_crs(26915)


# Limit parcels to just those in the desired township
#twp_parcels = parcels[parcels['TWP_CITY'] == twp]

parcels['Parcel_Township'] = parcels['TWP_CITY'].str.removesuffix(' TOWNSHIP')

verified_wells = gpd.read_file(verified_wells_path)
unverified_wells = gpd.read_file(unverified_wells_path)

verified_wells['Verified'] = True
unverified_wells['Verified'] = False

# Join verified and unverified wells into a single dataframe
wells = gpd.GeoDataFrame(pd.concat([verified_wells, unverified_wells], ignore_index=True))

# Export all wells
#wells.to_file(output_dir / "wells.gpkg")

In [ ]:

# Set samples variable to the output of the geocoding process
samples = out
samples_out = samples.copy()
samples_out.set_index('Lab_SampleID', inplace=True)
samples_out['Confidence_Code'] = pd.Series(dtype='int')


In [ ]:
# Address_1 field in parcels contains the RR address

# Join on ADDRESS_1, rr_addresses

samples = samples.replace('', pd.NA).dropna(subset=['lat', 'long'])
located_samples = gpd.GeoDataFrame(samples, geometry=gpd.points_from_xy(samples["long"], samples["lat"], crs="EPSG:4326")).to_crs(26915)
print(len(located_samples))
print(located_samples.crs)

located_samples.to_file("test_samples.gpkg")
located_samples = gpd.sjoin(parcels, located_samples, how="inner", predicate="intersects")
samples_out.loc[samples_out.index.isin(located_samples['Lab_SampleID']), 'Confidence_Code'] = -2
print(len(located_samples), "samples match at least one parcel")

# Set the right coordinate system (NAD27 UTM 15N, EPSG 26915)
located_samples = located_samples.to_crs(26915)
located_samples.to_file(output_dir / 'located_many.gpkg')

# Remove uncessary columns, drop cases where multiple parcels have the same RR address
multiple_parcels = located_samples[located_samples.duplicated(subset=['Lab_SampleID'], keep=False)]

multiple_parcels.to_file(output_dir / 'multiple_parcels.gpkg')
print(multiple_parcels["Lab_SampleID"])
located_samples = located_samples[['Lab_SampleID', 'SampleAddress', 'geometry']].drop_duplicates(subset='Lab_SampleID', keep=False)

print(len(multiple_parcels))

# Assign a confidence code of -2 to samples where multiple parcel polygons match the address
#samples_out.loc[multiple_parcels["Lab_SampleID"], 'Confidence_Code'] = -2
#print(len(samples_out[samples_out['Confidence_Code'] == -2]))

print(len(located_samples), "samples match exactly one parcel")
#samples_out.to_csv(output_dir / 'samples_out_test.csv')

#TODO: 134 in confidence code vs 291 here


In [ ]:
# Match wells to parcels with associated samples using an intersect test
sample_well_match = gpd.sjoin(located_samples, wells, how="inner", predicate="intersects")
print(len(sample_well_match))
sample_well_match.to_file(output_dir / 'sample_well_match.gpkg')

# Drop rows where the year sampled is before the year drilled
# unknown drill dates have a value of '0' and therefore survive this check

# Check if the sample ID contains a sample year, otherwise skip this check
date_checkable_samples = sample_well_match[sample_well_match['Lab_SampleID'].str.contains('\d{4}-\d{2}-\d{4}', regex=True)]
non_date_checkable_samples = sample_well_match[~sample_well_match['Lab_SampleID'].str.contains('\d{4}-\d{2}-\d{4}', regex=True)] # There's probably an easier way of just taking the difference between the last df and the df with everything in it
print(len(date_checkable_samples))
print(len(non_date_checkable_samples))

sample_well_date_match = date_checkable_samples[date_checkable_samples['Lab_SampleID'].str.slice(0,4) >= date_checkable_samples['DATE_DRLL'].astype(str).str.slice(0,4)]
invalid_dates = date_checkable_samples[date_checkable_samples['Lab_SampleID'].str.slice(0,4) < date_checkable_samples['DATE_DRLL'].astype(str).str.slice(0,4)]
print(len(invalid_dates))
# Assign a confidence code of -4 for samples where the only wells on the matching property were drilled after the sample date
samples_out.loc[invalid_dates['Lab_SampleID'], 'Confidence_Code'] = -4

passed_samples = pd.concat([sample_well_date_match, non_date_checkable_samples])
print(len(passed_samples))

sample_well_date_match.to_file(output_dir / 'sample_well_match_datechecked.gpkg')


# Assign a confidence code of -3 to samples where multiple wells may match the sample
multiple_wells = passed_samples[passed_samples.duplicated(subset=['Lab_SampleID'], keep=False)]
samples_out.loc[multiple_wells['Lab_SampleID'], 'Confidence_Code'] = -3 
print(len(passed_samples), "Hi")

# Filter to only necessary columns and remove cases where there are multiple wells on a parcel
unique_sample_well_match = passed_samples[['Lab_SampleID', 'UTME', 'UTMN', 'WELLID', 'Verified', 'DATE_DRLL', 'geometry']].drop_duplicates(subset='Lab_SampleID', keep=False)
print(len(unique_sample_well_match))

In [ ]:

# Export samples that can be confidently matched to a single well
unique_sample_well_match.to_file(output_dir / 'unique_sample_well_match_911.gpkg')
unique_sample_well_match.to_csv(output_dir / 'unique_sample_well_match_911.csv')

# Assign a confidence code of 1 to samples that match with the highest confidence
# Assign a confidence code of 2 to samples that may match, but have an unknown drill date
unknown_date = unique_sample_well_match.loc[unique_sample_well_match['DATE_DRLL'] == 0]
samples_out.loc[unique_sample_well_match['Lab_SampleID'], 'Confidence_Code'] = 1
samples_out.loc[unknown_date['Lab_SampleID'], 'Confidence_Code'] = 2
samples_out.to_csv(output_dir / 'samples_out_test.csv')



print("Matches with at least one well:", len(sample_well_match))
print("Matches with exactly one well:", len(unique_sample_well_match))